# Creating a RAG with an In-Memory Vector Database

- Models: OpenAI models or Ollama Open-Source models
- Databases: In-Memory Vector Database
- Data type: PDF

## Content

- [Running an LLM](#Running-an-LLM)
- [Loading PDF Documents](#Loading-PDF-Documents)
- [Prompt Engineering](#Prompt-Engineering)
- [Creating a Local In-Memory Vector Database](#Creating-a-Local-In-Memory-Vector-Database)
- [Testing End-to-End RAG](#Testing-End-to-End-RAG)

In [ ]:
from utils import (DocArrayInMemorySearch, Path, PromptTemplate, PyPDFLoader,
                   StrOutputParser, itemgetter, load_embedding_model,
                   load_model)

### Running an LLM

In [2]:
# MODEL = "llama3"
# MODEL = "phi3:mini"
MODEL = "gpt-3.5-turbo"
# MODEL_EMBEDDINGS = "text-embeddings-3-small"

model = load_model(MODEL)
embeddings = load_embedding_model(source="openai", name=None)

In [3]:
model.invoke("Hola, ¿cómo estás?")

AIMessage(content='¡Hola! Estoy aquí para ayudarte, ¿en qué puedo asistirte hoy?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5be6482c-1357-4491-afb9-a27d1c89f3b8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 20, 'total_tokens': 35})

In [4]:
parser = StrOutputParser()

chain = model | parser # For llama3, this is not necessary, but it is for other models
chain.invoke("Explica el concepto de democracia")

### Loading PDFs Documents

In [15]:
file_path = Path(".") / "database" / "pdfs" / "doc1.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
pages

[Document(page_content='CONGRESO DE LOS DIPUTADOS\nXV LEGISLATURA\nSerie A:  \nPROYECTOS DE LEY  24 de mayo de 2024  Núm. 21-1  Pág. 1BOLETÍN OFICIAL  \nDE LAS CORTES GENERALES\nPROYECTO DE LEY\n121/000021  Proyecto de Ley de universalidad del Sistema Nacional \nde Salud. \nLa Mesa de la Cámara, en su reunión del día de hoy, ha adoptado el acuerdo que se \nindica respecto del asunto de referencia.\n(121) Proyecto de ley.\nAutor: Gobierno.\nProyecto de Ley de universalidad del Sistema Nacional de Salud.\nAcuerdo:\nEncomendar su aprobación con competencia legislativa plena, conforme al artículo 148 del \nReglamento, a la Comisión de Sanidad. Asimismo, publicar en el Boletín Oficial de las \nCortes Generales, estableciendo plazo de enmiendas, por un período de quince días \nhábiles, que finaliza el día 11 de junio de 2024.\nEn ejecución de dicho acuerdo se ordena la publicación de conformidad con el \nartículo 97 del Reglamento de la Cámara.\nPalacio del Congreso de los Diputados, 21 de m

### Prompt Engineering

In [16]:

template = """
Responde a la siguiente pregunta basándote en el contexto a continuación. Si no puedes responder, escribe "No sé".

Contexto: {context}

Pregunta: {question}
"""

prompt = PromptTemplate.from_template(template)

In [17]:
print(prompt.format(context="Me llamo Gonzalo.", question="¿Cómo me llamo?"))


Responde a la siguiente pregunta basándote en el contexto a continuación. Si no puedes responder, escribe "No sé".

Contexto: Me llamo Gonzalo.

Pregunta: ¿Cómo me llamo?



In [18]:
chain = prompt | model | parser

chain.invoke(
    {
        "context": "Me llamo Gonzalo.",
        "question": "¿Cómo me llamo?"
    }
)

'Te llamas Gonzalo.'

In [19]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

### Creating a Local In-Memory Vector Database

In [20]:
# LOCAL IN-MEMORY VECTOR STORE
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)
retriever = vectorstore.as_retriever()
retriever.invoke("puntos clave de esta ley")


[Document(page_content='acorde con el principio de proporcionalidad al contener la regulación imprescindible para \nla consecución de los objetivos previamente mencionados, ya que las medidas que ahora \nse regulan resultan proporcionadas al bien público que se trata de proteger, no restringe \nderechos ni impone obligaciones y supone una extensión del ámbito de aplicación del \nderecho a la protección de la salud y de su alcance. Igualmente, se ajusta al principio \ncve: BOCG-15-A-21-1', metadata={'source': 'database/pdfs/doc1.pdf', 'page': 6}),
 Document(page_content='BOLETÍN OFICIAL DE LAS CORTES GENERALES\nCONGRESO DE LOS DIPUTADOS\nSerie A  Núm. 21-1  24 de mayo de 2024  Pág. 8\nde seguridad jurídica, siendo coherente con el resto del ordenamiento jurídico. Asimismo, \ncumple con el principio de transparencia, ya que identifica claramente su propósito y \nse ofrece una explicación. Y en relación con el principio de eficiencia, esta ley no impone \ncarga administrativa alguna adici

### Testing End-to-End RAG

In [21]:

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [22]:
chain.invoke(
    {
        "question": "¿Cuándo entrará en vigor el proyecto de ley?"
    }
)

'El proyecto de ley entrará en vigor a los veinte días de su publicación en el Boletín Oficial del Estado.'

In [23]:
chain.invoke(
    {
        "question": "Cuál es el artículo 14 del proyecto de ley?"
    }
)

'El artículo 14 del proyecto de ley se refiere a la prestación de atención sociosanitaria.'

In [24]:
# To see the output token by token as it is generated
# for s in chain.stream({"question": "¿Cuál es el artículo 14 del proyecto de ley?"}):
#     print(s, end="", flush=True)

In [25]:
# # Batch processing
# questions = []
# chain.batch([{"question" : q} for q in questions])

**New document**

In [26]:
file_path = Path(".") / "database" / "pdfs" / "doc2.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)
retriever = vectorstore.as_retriever()
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [28]:
questions = [
    "Detalla el título, la fecha, el autor y todos los stakeholders de esta proposición de ley.",
    "De forma muy resumida, ¿por qué es importante esta ley?",
    "De forma muy resumida, ¿a quién afecta esta ley?",
    "Lista y describe los puntos clave de esta ley."
]

In [29]:
for q in questions:
    print(f"### Question: {q}")
    print(f"### Answer: {chain.invoke({'question': q})}")
    print()

### Question: Detalla el título, la fecha, el autor y todos los stakeholders de esta proposición de ley.
### Answer: Título: Proposición de Ley por la que se modifica la Ley 12/2013, de 2 de agosto, de medidas para mejorar el funcionamiento de la cadena alimentaria.
Fecha: 15 de marzo de 2024
Autor: Grupo Parlamentario Republicano
Stakeholders:
- Grupo Parlamentario Republicano
- Portavoz adjunta del Grupo Parlamentario Teresa Jordà i Roura
- Ministerio competente en materia de agricultura, ganadería y alimentación

### Question: De forma muy resumida, ¿por qué es importante esta ley?
### Answer: Esta ley es importante porque busca mejorar el funcionamiento de la cadena alimentaria para garantizar precios justos a los productores agrícolas y combatir prácticas desleales en la distribución de alimentos. También busca proteger a los agricultores de vender a pérdidas y establecer un marco normativo que promueva la transparencia y equilibrio en la cadena alimentaria.

### Question: De form